1. RFE(Recursive Feature Elimination)
    우선 모든 변수를 사용하여 모델을 훈련, 가장 중요하지 않은 변수를 제거
    
    원하는 변수의 수에 도달할 때까지 과정을 반복
    
    변수의 중요성은 의사 결정 나무의 변수 중요도 또는 선형 모델의 계수값, 신경망의 가중치의 계수 크기로 어떤 변수를 삭제해야할지 결정

2. RFE-CV (Cross Validation 교차 검증을 도입)

3. UFS (특정 검전 통계(F-Value), 또는 카이-제곱 값)을 계산하여 각 변수를 평가하고, 가장 높은 점수를 가진 변수 선택

데이터 전처리 및 어떤 변수를 제거해야할지 파악해보자..

# 전처리에 필요한 라이브러리 및 함수 가져오기와 머신 러닝 함수 가져오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import f1_score
warnings.filterwarnings('ignore')

# 데이터 가져오기

In [2]:
df=pd.read_csv(r'C:\Users\hyssk\AICE_\Dacon_DataAnlysis\train.csv')
df.drop('user_id',axis=1,inplace=True) # 과대적합을 야기할 user_id 삭제
df

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern,target
0,13,14,14.946163,8.427187,18,16,68.360455,3,4,Low,Basic,4,5,0
1,16,18,18.453224,72.646087,16,13,97.567322,2,3,Medium,Basic,1,6,1
2,22,1,16.195228,21.774492,13,14,94.358763,3,4,Medium,Premium,0,7,1
3,1,19,17.628656,42.659066,19,18,70.153228,0,3,Low,Basic,1,0,1
4,4,5,21.390656,30.744287,19,10,81.917908,2,4,Medium,Basic,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,22,29,14.727623,84.053558,18,16,64.966803,2,5,Low,Premium,1,1,1
9996,10,11,19.374054,45.464833,9,8,82.750244,3,3,Medium,Basic,2,7,1
9997,7,27,18.240978,127.302411,24,14,81.567839,3,5,High,Basic,1,6,1
9998,11,7,18.783800,5.297234,10,10,89.885656,4,5,Low,Basic,2,0,1


# df에 결측치가 존재하는지 파악

In [3]:
df.isnull().sum()

subscription_duration                0
recent_login_time                    0
average_login_time                   0
average_time_per_learning_session    0
monthly_active_learning_days         0
total_completed_courses              0
recent_learning_achievement          0
abandoned_learning_sessions          0
community_engagement_level           0
preferred_difficulty_level           0
subscription_type                    0
customer_inquiry_history             0
payment_pattern                      0
target                               0
dtype: int64

결측치가 존재하지 않는다.

# 범주형 데이터 정제하기

레이블
- Pandas (Factorize)
- sklearn (LabelEncoder)

원핫 인코딩
- pd.get_dummies(하고자하는 칼럼명)
- from sklearn.preprocessing import OneHotEncoder

In [4]:
# 범주형 데이터 전처리 (preferred_difficulty_level) factorize 코딩
labeled_preferred_difficulty_level=pd.factorize(df['preferred_difficulty_level'])[0].reshape(-1,1)
df.insert(9,'labeled_preferred_difficulty_level',labeled_preferred_difficulty_level)

In [5]:
df.drop('preferred_difficulty_level',axis=1,inplace=True)
df

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,labeled_preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern,target
0,13,14,14.946163,8.427187,18,16,68.360455,3,4,0,Basic,4,5,0
1,16,18,18.453224,72.646087,16,13,97.567322,2,3,1,Basic,1,6,1
2,22,1,16.195228,21.774492,13,14,94.358763,3,4,1,Premium,0,7,1
3,1,19,17.628656,42.659066,19,18,70.153228,0,3,0,Basic,1,0,1
4,4,5,21.390656,30.744287,19,10,81.917908,2,4,1,Basic,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,22,29,14.727623,84.053558,18,16,64.966803,2,5,0,Premium,1,1,1
9996,10,11,19.374054,45.464833,9,8,82.750244,3,3,1,Basic,2,7,1
9997,7,27,18.240978,127.302411,24,14,81.567839,3,5,2,Basic,1,6,1
9998,11,7,18.783800,5.297234,10,10,89.885656,4,5,0,Basic,2,0,1


In [6]:
cate=pd.Categorical(df['subscription_type'],['Basic','Premium'],ordered=True)
labels,unique=pd.factorize(cate,sort=True)
df['subscription_type']=labels
df

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,labeled_preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern,target
0,13,14,14.946163,8.427187,18,16,68.360455,3,4,0,0,4,5,0
1,16,18,18.453224,72.646087,16,13,97.567322,2,3,1,0,1,6,1
2,22,1,16.195228,21.774492,13,14,94.358763,3,4,1,1,0,7,1
3,1,19,17.628656,42.659066,19,18,70.153228,0,3,0,0,1,0,1
4,4,5,21.390656,30.744287,19,10,81.917908,2,4,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,22,29,14.727623,84.053558,18,16,64.966803,2,5,0,1,1,1,1
9996,10,11,19.374054,45.464833,9,8,82.750244,3,3,1,0,2,7,1
9997,7,27,18.240978,127.302411,24,14,81.567839,3,5,2,0,1,6,1
9998,11,7,18.783800,5.297234,10,10,89.885656,4,5,0,0,2,0,1


In [7]:
df

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,labeled_preferred_difficulty_level,subscription_type,customer_inquiry_history,payment_pattern,target
0,13,14,14.946163,8.427187,18,16,68.360455,3,4,0,0,4,5,0
1,16,18,18.453224,72.646087,16,13,97.567322,2,3,1,0,1,6,1
2,22,1,16.195228,21.774492,13,14,94.358763,3,4,1,1,0,7,1
3,1,19,17.628656,42.659066,19,18,70.153228,0,3,0,0,1,0,1
4,4,5,21.390656,30.744287,19,10,81.917908,2,4,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,22,29,14.727623,84.053558,18,16,64.966803,2,5,0,1,1,1,1
9996,10,11,19.374054,45.464833,9,8,82.750244,3,3,1,0,2,7,1
9997,7,27,18.240978,127.302411,24,14,81.567839,3,5,2,0,1,6,1
9998,11,7,18.783800,5.297234,10,10,89.885656,4,5,0,0,2,0,1


In [8]:
ohe=OneHotEncoder()
a=ohe.fit_transform(df[['payment_pattern']]).toarray()

col=['3개월 모두 결제함','첫 2개월은 결제했으나 마지막 달에는 결제하지 않음','첫 달과 마지막 달에 결제함',
'첫 달에만 결제함','마지막 2개월에 결제함','가운데 달에만 결제함','마지막 달에만 결제함','3개월 동안 결제하지 않음']
col.reverse()
onehotencoded=pd.DataFrame(a,columns=[i for i in col])
df.drop(['payment_pattern'],axis=1,inplace=True)


In [17]:
pd.concat([df,onehotencoded],axis=1)

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,labeled_preferred_difficulty_level,...,customer_inquiry_history,target,3개월 동안 결제하지 않음,마지막 달에만 결제함,가운데 달에만 결제함,마지막 2개월에 결제함,첫 달에만 결제함,첫 달과 마지막 달에 결제함,첫 2개월은 결제했으나 마지막 달에는 결제하지 않음,3개월 모두 결제함
0,13,14,14.946163,8.427187,18,16,68.360455,3,4,0,...,4,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,16,18,18.453224,72.646087,16,13,97.567322,2,3,1,...,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,22,1,16.195228,21.774492,13,14,94.358763,3,4,1,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,19,17.628656,42.659066,19,18,70.153228,0,3,0,...,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,21.390656,30.744287,19,10,81.917908,2,4,1,...,3,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,22,29,14.727623,84.053558,18,16,64.966803,2,5,0,...,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,10,11,19.374054,45.464833,9,8,82.750244,3,3,1,...,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9997,7,27,18.240978,127.302411,24,14,81.567839,3,5,2,...,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9998,11,7,18.783800,5.297234,10,10,89.885656,4,5,0,...,2,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# feature와 target 나누기


In [10]:
X=df.drop(['target'],axis=1)
y=df['target']

In [13]:
# train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [14]:
len(X_train.keys())
X_train.shape

(8000, 12)

# Scaler (Normalization | Standarzation)

칼럼마다 값의 scale이 다르다.

각 칼럼의 상대적인 크기에 따라 모델 성능에 크게 영향을 끼칠 가능성이 있기 때문에, 

Scaler를 적용할 필요성이 있다.

In [ ]:
scaled_column=['subscription_duration','recent_login_time','average_login_time','monthly_active_learning_days','total_completed_courses','recent_learning_achievement'] # 스케일될 수치 칼럼들 experiment 1


In [15]:
# Decision tree
dt=DecisionTreeClassifier(random_state=1001,
                          max_depth=2
                          )

# 의사결정나무 학습하기
dt_model=dt.fit(X_train,y_train)
dt_model.predict()
# 학습데이터 정확도확인
print("학습 정확도: ",dt_model.score(X_train,y_train))

# 검증데이터 정확도확인
print("검증 정확도: ",dt_model.score(X_test,y_test))

dt_model.feature_importances_

for i in range(len(dt_model.feature_importances_)):
  print(f"{X_train.columns[i]} = {dt_model.feature_importances_[i]}")


TypeError: predict() missing 1 required positional argument: 'X'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import random
import numpy as np


In [ ]:
# 모델 시드 고정하기
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

model=Sequential()
model.add(Dense(32,activation='relu',input_shape=(13,)))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='SGD',metrics=['accuracy'])

model.summary()


In [ ]:
es=EarlyStopping(monitor='val_loss',min_delta=0,patience=10,verbose=1,restore_best_weights=True)
history=model.fit(X_train,y_train,epochs=100,batch_size=128,verbose=1,validation_data=(X_test,y_test),callbacks=[es])


In [ ]:
import matplotlib.pyplot as plt

# 훈련 과정 정확도 시각화하기
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(['Train',"Validation"],loc='lower right')
plt.show()

# 훈련 과정 손실 시각화
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Train, Validation"],loc='upper right')
plt.show()